# Total precipitation mean, stdm and quantiles

In [1]:
import os
import sys
import yaml
from glob import glob
from datetime import datetime, timedelta

import numpy as np
import xarray as xr
import pandas as pd

In [2]:
import matplotlib.pyplot as plt
%matplotlib inline

In [3]:
sys.path.insert(0, os.path.realpath('../libs/'))
import verif_utils as vu
import seeps_utils as seeps

In [4]:
config_name = os.path.realpath('verif_config.yml')

with open(config_name, 'r') as stream:
    conf = yaml.safe_load(stream)

In [5]:
model_name = 'fuxi_physics'
lead_range = conf[model_name]['lead_range']
verif_lead = [120,]
leads_exist = list(np.arange(lead_range[0], lead_range[-1]+lead_range[0], lead_range[0]))
ind_lead = vu.lead_to_index(leads_exist, verif_lead)

In [6]:
path_verif = conf[model_name]['save_loc_verif']+'TP_stats_{:03d}h_{}.nc'.format(
    verif_lead[0], model_name)

## Verification setup

In [7]:
# ---------------------------------------------------------------------------------------- #
# forecast
filename_OURS = sorted(glob(conf[model_name]['save_loc_gather']+'*.nc'))

# pick years
year_range = conf[model_name]['year_range']
years_pick = np.arange(year_range[0], year_range[1]+1, 1).astype(str)
filename_OURS = [fn for fn in filename_OURS if any(year in fn for year in years_pick)]

In [8]:
variable_levels = {
    'total_precipitation': None,
}

In [9]:
# ---------------------------------------------------------------------------------------- #
# RMSE compute
tp_collect = []

for fn_ours in filename_OURS:
    # detect 00Z vs 12Z
    ini = int(fn_ours[-6:-4])
    
    ds_ours = xr.open_dataset(fn_ours)
    ds_ours = ds_ours.isel(time=ind_lead)
    ds_ours = vu.ds_subset_everything(ds_ours, variable_levels)
    
    # ------------------------------------------- #
    # convert neg precip to 0 before accumulation 
    ds_ours['total_precipitation'] = xr.where(
        ds_ours['total_precipitation'] < 0, 0, ds_ours['total_precipitation']
    )
    ds_ours_24h = vu.accum_6h_24h(ds_ours, ini)
    #ds_ours_24h = ds_ours_24h.compute()
    
    tp_collect.append(ds_ours_24h)
    
# Combine verif results
ds_all_24h = xr.concat(tp_collect, dim='time')

In [10]:
ds_mean = ds_all_24h['total_precipitation'].mean(dim='time')
ds_std = ds_all_24h['total_precipitation'].std(dim='time')
ds_p95 = ds_all_24h['total_precipitation'].quantile(0.95, dim='time')

# Combine these into a new dataset
ds_summary = xr.Dataset(
    {
    'mean': ds_mean,
    'std': ds_std,
    'p95': ds_p95
})

In [11]:
path_verif

'/glade/campaign/cisl/aiml/ksha/CREDIT_physics/VERIF/fuxi_plevel_physics/TP_stats_120h_fuxi_physics.nc'

In [12]:
# ds_summary.to_netcdf(path_verif)